In [2]:
import pandas as pd
import matplotlib.pyplot  as plt
from jqdatasdk import *
import numpy as np
import talib as tl #技术指标
import datetime as dt
auth('15652961415','715905Si')
#sns.set_style('darkgrid')
plt.rcParams['font.sans-serif']=['SimHei']
path_base = 'C:/wangxiaolei/quant/Rong360-master/hongguan/'
#-*- coding: utf-8 -*-

auth success 


In [3]:
def show_index_hs(df_index, label):
    df_hs = get_price('000300.XSHG',start_date='2006-01-01',end_date='2018-11-30',fields='close')
    fig = plt.figure(figsize=(15,6))
    x_index = df_hs.index
    ax1 = fig.add_subplot(111)
    ax1.plot(df_index,color = 'black', linewidth=2, label=label)
    ax1.legend(loc=2,fontsize=12)
    ax2=ax1.twinx()#这是双坐标关键一步
    ax2.plot(df_hs['close'], color = 'red',linewidth=2,label='沪深300指数')
    ax2.legend(loc=1,fontsize=12)
    # ax1.grid(False)
    ax2.grid(False)
    plt.show()
def show_benifit(df, label):
    x_index = df.index
    plt.figure(figsize=(12,6))
    plt.plot(x_index, df['benchmark'], color='black', label='benchmark')
    plt.plot(x_index, df['net_value'], color='red', label='net_value')
    plt.axhline(0,ls='--')
    plt.title(label)
    plt.xlabel('日期')
    plt.ylabel('收益')
    plt.legend()
df_hs = get_price('000300.XSHG',start_date='2006-01-01',end_date='2018-11-30',fields='close')
df_hs_M = df_hs.resample('M',how='last')
index = pd.date_range(df_hs.index[0], df_hs.index[-1])
rate_riskfree = 0.0 #无风险收益

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()


In [6]:
#一、pmi指标
#经理购物指标，指标周期3个月，因为pmi指标并非平滑，以一个季度为粒度并进行平均均值
#pmi信号：3月均线上涨即认为为买入信号
path_pmi = path_base + 'PMI.xls'
pmi_df = pd.read_excel(path_pmi,sheetname= [0])[0]
pmi_df.set_index('日期', inplace = True)
#pmi_df.plot(figsize=(15,6),title='PMI')
#获取当前日期过去三个月PMI均值，与上月此时过去三个月PMI均值对比，若是PMI上涨并且本月PMI值大于50则认为本月可以买入指数
pmi_df['pmi_3_mean'] = pmi_df['PMI'].rolling(window=3).mean()
pmi_df['pmi_3_mean_shift_1'] = pmi_df['pmi_3_mean'].shift(1)
#pmi_df['position']=((pmi_3_mean > pmi_3_mean_shift_1) * 1) * ((pmi_3_mean.map(lambda x:True if x > 50 else False)) * 1)
pmi_df['signal'] = (pmi_df['pmi_3_mean'] > pmi_df['pmi_3_mean_shift_1']) * 1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [7]:
#二、利率指标
#利率择时,低利率买进，高利率卖出，以布尔轨道为依据
path_rate = path_base + 'SHIBOR.xls'
df_shibor = pd.read_excel(path_rate,sheetname= [0])[0]
df_shibor.set_index('日期', inplace = True)
df_hs =  get_price('000300.XSHG',start_date = df_shibor.index[0], end_date = df_shibor.index[-1],fields='close')
df_hs_M = df_hs.resample('M',how='last')
upperband, middleband, lowerband = (tl.BBANDS(df_shibor[df_shibor.columns[1]].values, timeperiod=12, nbdevup=1.8, nbdevdn=1.8))
df_shibor['BBAND_upper'] = upperband
df_shibor['BBAND_middle'] = middleband
df_shibor['BBAND_lower'] = lowerband
signal = 0 #中间状态
for date in df_shibor.index:
    if df_shibor.loc[date, df_shibor.columns[1]] > df_shibor.loc[date,'BBAND_middle']:
        df_shibor.loc[date, 'signal'] = 0
    elif df_shibor.loc[date, df_shibor.columns[1]] < df_shibor.loc[date,'BBAND_lower']:
        df_shibor.loc[date, 'signal'] = 1
    else:
        df_shibor.loc[date, 'signal'] = signal
    signal = df_shibor.loc[date, 'signal']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  import sys


In [8]:
#三、国债利差指标
#获取国债期限利差数据
gz_path = path_base + '/guozhai.xls'
df_gz_spread = pd.read_excel(gz_path, sheet_name= [0])[0]
df_gz_spread.set_index('日期',inplace=True)
df_gz_spread = df_gz_spread.fillna(method='ffill')
df_hs =  get_price('000300.XSHG',start_date = df_gz_spread.index[0], end_date = df_gz_spread.index[-1],fields='close').dropna()
df_gz_spread = df_gz_spread.reindex(df_hs.index)
df_hs_M = df_hs.resample('M',how='last')
df_gz_spread['term_spread_10_1'] = df_gz_spread['中债国债到期收益率:10年'] - df_gz_spread['中债国债到期收益率:1个月']
upperband, middleband, lowerband = (tl.BBANDS(df_gz_spread['term_spread_10_1'].values, timeperiod=25, nbdevup=1.8, nbdevdn=1.8))
df_gz_spread['BBAND_upper'] = upperband
df_gz_spread['BBAND_middle'] = middleband
df_gz_spread['BBAND_lower']=lowerband
signal = 0
for date in df_gz_spread.index:
    if df_gz_spread.loc[date,'term_spread_10_1'] < df_gz_spread.loc[date,'BBAND_middle']:
        df_gz_spread.loc[date,'signal'] = 0
    elif df_gz_spread.loc[date,'term_spread_10_1'] > df_gz_spread.loc[date,'BBAND_upper']:
        df_gz_spread.loc[date,'signal'] = 1.0
    else:
        df_gz_spread.loc[date,'signal'] = signal
    signal = df_gz_spread.loc[date, 'signal']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  if __name__ == '__main__':


In [10]:
#四、信用利差
#获取企业债和国债利差数据
gz_path = path_base + '/guozhai.xls'
df_gz = pd.read_excel(gz_path, sheet_name= [0])[0]
df_gz.set_index('日期',inplace=True)
df_gz = df_gz.fillna(method='ffill')
path_qyz = path_base +'qiyezhai.xls'
df_qyz = pd.read_excel(path_qyz, sheet_name = [0])[0]
df_qyz.set_index('日期',inplace=True)
df_qyz = df_qyz.fillna(method='ffill')
df_qyz['credit_spread'] = df_qyz['中债企业债到期收益率(AAA):1个月'] - df_gz['中债国债到期收益率:1个月']
df_hs =  get_price('000300.XSHG',start_date = df_qyz.index[0], end_date = df_qyz.index[-1],fields='close').dropna()
df_qyz = df_qyz.reindex(df_hs.index)
upperband, middleband, lowerband = (tl.BBANDS(df_qyz['credit_spread'].values, timeperiod=25, nbdevup=1.8, nbdevdn=1.8))
df_qyz['BBAND_upper'] = upperband
df_qyz['BBAND_middle'] =  middleband
df_qyz['BBAND_lower'] = lowerband
signal = 0
for date in df_qyz.index:
    if df_qyz.loc[date,'credit_spread'] > df_qyz.loc[date,'BBAND_middle']:
        df_qyz.loc[date,'signal']=0
    elif df_qyz.loc[date,'credit_spread'] < df_qyz.loc[date,'BBAND_lower']:
        df_qyz.loc[date,'signal']=1.0
    else:
        df_qyz.loc[date,'signal']=signal
    signal=df_qyz.loc[date,'signal']